In [1]:
import os

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import pandas as pd


import cv2
import math

In [2]:
# Config
model_path = './models/pose_landmarker_heavy.task'
video_folder = '/home/ruben/Downloads/UBnormal'

In [3]:
def draw_landmarks_on_image(rgb_image, detection_result):
    pose_landmarks_list = detection_result.pose_landmarks
    annotated_image = np.copy(rgb_image)

    # Loop through the detected poses to visualize.
    for idx in range(len(pose_landmarks_list)):
        pose_landmarks = pose_landmarks_list[idx]

        # Draw the pose landmarks.
        pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        pose_landmarks_proto.landmark.extend([
          landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
          annotated_image,
          pose_landmarks_proto,
          solutions.pose.POSE_CONNECTIONS,
          solutions.drawing_styles.get_default_pose_landmarks_style())
    return annotated_image

In [4]:
def generate_graph_matrix(landmarks):
    """Generate a matrix representing the graph. The matrix values is the distance between the different graphs
    """
    n_markers = len(landmarks)
    graph_matrix = []
    for _ in range(n_markers):
        graph_matrix.append([0] * n_markers)
    co_list = [(lm.x, lm.y, lm.z) for lm in landmarks]
    for e_1 in range(n_markers):
        co_1 = co_list[e_1]
        for e_2 in range(n_markers):
            co_2 = co_list[e_2]
            if e_1 != e_2:
                dist = math.dist(co_1, co_2)
                graph_matrix[e_1][e_2] = dist
    return graph_matrix

In [5]:
# Inspiration: https://stackoverflow.com/questions/71319286/iterate-through-directories-subdirectories-to-get-files-with-specialized-ext
def get_video_files(directory: str):
    """Function that exctracts the mp4 files from the given directory
    and returns the path to the video and it's annotations
    """
    filelist = [];
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.mp4'):
                video_name = file.split('.mp4')[0]
                video_file = os.path.join(root,file)
                annotation_file = os.path.join(root, f'{video_name}_annotations/{video_name}_tracks.txt')
                filelist.append((video_file, annotation_file))
    return filelist

def get_annotations(annotation_file: str):
    """Function that fetches the annotations and converts them to a usable format
    """
    file=open(annotation_file,"r")
    annotations = [line.strip() for line in file.readlines()]
    file.close()
    parsed_annotations = []
    for item in annotations:
        object_id, start_frame, end_frame = [int(i) for i in item.split(',')] # TODO: check if we need annotation
        parsed_annotations.append((object_id, start_frame, end_frame))
    return parsed_annotations

In [6]:
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a pose landmarker instance with the video mode:
options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO)

In [7]:
# Inspiration: https://medium.com/@riddhisi238/real-time-pose-estimation-from-video-using-mediapipe-and-opencv-in-python-20f9f19c77a6
# https://developers.google.com/mediapipe/solutions/vision/pose_landmarker/python#handle_and_display_results

video_files = get_video_files(video_folder)
print(f"Number of video's found: {len(video_files)}")

poses = []

for index, item in enumerate(video_files):
    video_file, annotation_file = item
    print(f'Processing video {index}: {video_file.split("/")[-1]}')    
    # annotations = get_annotations(annotation_file)
    
    video_name = video_file.split('/')[-1]
    if 'abnormal' in video_name:
        label = 'abnormal'
    else:
        label = 'normal'
    
    video = cv2.VideoCapture(video_file)
    fps = video.get(cv2.CAP_PROP_FPS)
    ms_per_frame = (1000/ fps) 

    with PoseLandmarker.create_from_options(options) as landmarker:
        n_frame = 0
        while video.isOpened():
            ret, frame = video.read()  # Ret: frame is correctly read or not
            if not ret:
                break
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
            frame_timestamp_ms = int(n_frame * ms_per_frame)
            pose_landmarker_result = landmarker.detect_for_video(mp_image, frame_timestamp_ms)
            landmarks = pose_landmarker_result.pose_landmarks
            if landmarks:
                for landmark_list in landmarks:
                    co_list = [(lm.x, lm.y, lm.z) for lm in landmark_list]
                    graph_matrix = generate_graph_matrix(landmark_list)
                    poses.append({
                        'video': video_name,
                        'frame': n_frame,
                        'label': label,
                        'markers': co_list,
                        'graph': graph_matrix
                    })
            n_frame += 1   

Number of video's found: 543
Processing video 0: normal_scene_20_scenario_5.mp4


I0000 00:00:1712158986.921296    6099 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712158986.925846    6185 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Processing video 1: abnormal_scene_20_scenario_6.mp4


I0000 00:00:1712159040.615012    6099 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712159040.616258    6246 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)


Processing video 2: abnormal_scene_20_scenario_8.mp4


...

I0000 00:00:1712174826.496617    6099 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712174826.497804   35293 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)


Processing video 541: normal_scene_29_scenario_1.mp4


I0000 00:00:1712174856.997656    6099 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712174856.998862   35345 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)


Processing video 542: abnormal_scene_29_scenario_2.mp4


I0000 00:00:1712174898.806150    6099 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712174898.807591   35400 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)


In [11]:
import pandas as pd
pd.DataFrame(poses).to_csv('./ubnormal_data.csv')

In [14]:
# Inspiration: https://medium.com/@riddhisi238/real-time-pose-estimation-from-video-using-mediapipe-and-opencv-in-python-20f9f19c77a6
# https://developers.google.com/mediapipe/solutions/vision/pose_landmarker/python#handle_and_display_results

# Testing
video_file = f'{video_folder}/Scene1/abnormal_scene_1_scenario_1.mp4'
annotation_file = f'{video_folder}/Scene1/abnormal_scene_1_scenario_1_annotations/abnormal_scene_1_scenario_1_tracks.txt'

video = cv2.VideoCapture(video_file)
fps = video.get(cv2.CAP_PROP_FPS)
ms_per_frame = (1000/ fps) 

video_name = video_file.split('/')[-1]
if 'abnormal' in video_name:
    label = 'abnormal'
else:
    label = 'normal'
poses = []

with PoseLandmarker.create_from_options(options) as landmarker:
    n_frame = 0
    while video.isOpened():
        ret, frame = video.read()  # Ret: frame is correctly read or not
        if not ret:
            break
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        frame_timestamp_ms = int(n_frame * ms_per_frame)
        pose_landmarker_result = landmarker.detect_for_video(mp_image, frame_timestamp_ms)
        landmarks = pose_landmarker_result.pose_landmarks
        if landmarks:
            for landmark_list in landmarks:
                graph_matrix = generate_graph_matrix(landmark_list)
                poses.append({
                    'video': video_name,
                    'frame': n_frame,
                    'label': label,
                    'graph': graph_matrix
                })
                
        n_frame += 1

I0000 00:00:1712074266.923086    4932 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1712074266.924502    5386 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)
